[【チュートリアル】機械学習を使って30分で固有表現抽出器を作る](https://qiita.com/Hironsan/items/326b66711eb4196aa9d4)

In [2]:
!pip install python-crfsuite


     |████████████████████████████████| 743 kB 2.8 MB/s eta 0:00:01


In [3]:
from itertools import chain
import pycrfsuite
import sklearn
from sklearn.metrics import classification_report
from sklearn.preprocessing import LabelBinarizer

In [4]:
import codecs

class CorpusReader(object):

    def __init__(self, path):
        with codecs.open(path, encoding='utf-8') as f:
            sent = []
            sents = []
            for line in f:
                if line == '\n':
                    sents.append(sent)
                    sent = []
                    continue
                morph_info = line.strip().split('\t')
                sent.append(morph_info)
        train_num = int(len(sents) * 0.9)
        self.__train_sents = sents[:train_num]
        self.__test_sents = sents[train_num:]

    def iob_sents(self, name):
        if name == 'train':
            return self.__train_sents
        elif name == 'test':
            return self.__test_sents
        else:
            return None

In [5]:
c = CorpusReader('motoki.txt')
train_sents = c.iob_sents('train')
test_sents = c.iob_sents('test')

In [6]:
def is_hiragana(ch):
    return 0x3040 <= ord(ch) <= 0x309F

def is_katakana(ch):
    return 0x30A0 <= ord(ch) <= 0x30FF

def get_character_type(ch):
    if ch.isspace():
        return 'ZSPACE'
    elif ch.isdigit():
        return 'ZDIGIT'
    elif ch.islower():
        return 'ZLLET'
    elif ch.isupper():
        return 'ZULET'
    elif is_hiragana(ch):
        return 'HIRAG'
    elif is_katakana(ch):
        return 'KATAK'
    else:
        return 'OTHER'

def get_character_types(string):
    character_types = map(get_character_type, string)
    character_types_str = '-'.join(sorted(set(character_types)))

    return character_types_str

In [7]:
def extract_pos_with_subtype(morph):
    idx = morph.index('*')

    return '-'.join(morph[1:idx])

In [8]:
def word2features(sent, i):
    word = sent[i][0]
    chtype = get_character_types(sent[i][0])
    postag = extract_pos_with_subtype(sent[i])
    features = [
        'bias',
        'word=' + word,
        'type=' + chtype,
        'postag=' + postag,
    ]
    if i >= 2:
        word2 = sent[i-2][0]
        chtype2 = get_character_types(sent[i-2][0])
        postag2 = extract_pos_with_subtype(sent[i-2])
        iobtag2 = sent[i-2][-1]
        features.extend([
            '-2:word=' + word2,
            '-2:type=' + chtype2,
            '-2:postag=' + postag2,
            '-2:iobtag=' + iobtag2,
        ])
    else:
        features.append('BOS')

    if i >= 1:
        word1 = sent[i-1][0]
        chtype1 = get_character_types(sent[i-1][0])
        postag1 = extract_pos_with_subtype(sent[i-1])
        iobtag1 = sent[i-1][-1]
        features.extend([
            '-1:word=' + word1,
            '-1:type=' + chtype1,
            '-1:postag=' + postag1,
            '-1:iobtag=' + iobtag1,
        ])
    else:
        features.append('BOS')

    if i < len(sent)-1:
        word1 = sent[i+1][0]
        chtype1 = get_character_types(sent[i+1][0])
        postag1 = extract_pos_with_subtype(sent[i+1])
        features.extend([
            '+1:word=' + word1,
            '+1:type=' + chtype1,
            '+1:postag=' + postag1,
        ])
    else:
        features.append('EOS')

    if i < len(sent)-2:
        word2 = sent[i+2][0]
        chtype2 = get_character_types(sent[i+2][0])
        postag2 = extract_pos_with_subtype(sent[i+2])
        features.extend([
            '+2:word=' + word2,
            '+2:type=' + chtype2,
            '+2:postag=' + postag2,
        ])
    else:
        features.append('EOS')

    return features


def sent2features(sent):
    return [word2features(sent, i) for i in range(len(sent))]


def sent2labels(sent):
    return [morph[-1] for morph in sent]


def sent2tokens(sent):
    return [morph[0] for morph in sent]

In [9]:
sent2features(train_sents[0])[0]

['bias',
 'word=履歴',
 'type=OTHER',
 'postag=名詞-一般',
 'BOS',
 'BOS',
 '+1:word=書',
 '+1:type=OTHER',
 '+1:postag=名詞-接尾-一般',
 '+2:word=について',
 '+2:type=HIRAG',
 '+2:postag=助詞-格助詞-連語']

In [10]:
X_train = [sent2features(s) for s in train_sents]
y_train = [sent2labels(s) for s in train_sents]

X_test = [sent2features(s) for s in test_sents]
y_test = [sent2labels(s) for s in test_sents]

In [11]:
trainer = pycrfsuite.Trainer(verbose=False)

for xseq, yseq in zip(X_train, y_train):
    trainer.append(xseq, yseq)

In [12]:
trainer.train('model.crfsuite')


In [13]:
tagger = pycrfsuite.Tagger()
tagger.open('model.crfsuite')

In [32]:
example_sent = test_sents[0]
print(' '.join(sent2tokens(example_sent)))

print("Predicted:", ' '.join(tagger.tag(sent2features(example_sent))))
print("Correct:  ", ' '.join(sent2labels(example_sent)))


x = sent2labels(example_sent)

for i in range(len(x)):
    if x[i]== 'B-PSN':
        s = sent2tokens(example_sent)
        print('名前：'+s[i])

AttributeError: 'Tagger' object has no attribute 'tag'

In [15]:
example_sent = test_sents[44]
print('入力データ： ' + ' '.join(sent2tokens(example_sent)))

x = sent2labels(example_sent)

for i in range(len(x)):
    if x[i]== 'B-PSN':
        s = sent2tokens(example_sent)
        print('名前：'+s[i]+ '(正解データ)')                

for i in range(len(x)):
    if x[i]== 'B-PSN':
        s = tagger.tag(sent2features(example_sent))
        print('名前：'+s[i]+ '(予測値)')        

IndexError: list index out of range

In [16]:
for i in range(100):
    example_sent = test_sents[i]
    print(' '.join(sent2tokens(example_sent)))

    print("Predicted:", ' '.join(tagger.tag(sent2features(example_sent))))
    print("Correct:  ", ' '.join(sent2labels(example_sent)))


Predicted: 
Correct:   


IndexError: list index out of range

In [17]:
example_sent = test_sents[0]
print(' '.join(sent2tokens(example_sent)))

print("Predicted:", ' '.join(tagger.tag(sent2features(example_sent))))
print("Correct:  ", ' '.join(sent2labels(example_sent)))


Predicted: 
Correct:   


# 履歴書 データ前処理

In [18]:
import MeCab
mytext = '履歴書ふりがなさとういちろう名前砂糖一郎2019年12月25日'
tagger = MeCab.Tagger()
print(tagger.parse(mytext))

t = tagger.parse(mytext)

履歴	名詞,一般,*,*,*,*,履歴,リレキ,リレキ
書	名詞,接尾,一般,*,*,*,書,ショ,ショ
ふりがな	名詞,一般,*,*,*,*,ふりがな,フリガナ,フリガナ
さとう	形容詞,自立,*,*,形容詞・アウオ段,連用ゴザイ接続,さとい,サトウ,サトー
いちろう	名詞,固有名詞,人名,名,*,*,いちろう,イチロウ,イチロー
名前	名詞,一般,*,*,*,*,名前,ナマエ,ナマエ
砂糖	名詞,一般,*,*,*,*,砂糖,サトウ,サトー
一	名詞,数,*,*,*,*,一,イチ,イチ
郎	名詞,一般,*,*,*,*,郎,ロウ,ロー
2019	名詞,数,*,*,*,*,*
年	名詞,接尾,助数詞,*,*,*,年,ネン,ネン
12	名詞,数,*,*,*,*,*
月	名詞,一般,*,*,*,*,月,ツキ,ツキ
25	名詞,数,*,*,*,*,*
日	名詞,接尾,助数詞,*,*,*,日,ニチ,ニチ
EOS



In [19]:
text = '	'.join(t.split(","))
text

'履歴\t名詞\t一般\t*\t*\t*\t*\t履歴\tリレキ\tリレキ\n書\t名詞\t接尾\t一般\t*\t*\t*\t書\tショ\tショ\nふりがな\t名詞\t一般\t*\t*\t*\t*\tふりがな\tフリガナ\tフリガナ\nさとう\t形容詞\t自立\t*\t*\t形容詞・アウオ段\t連用ゴザイ接続\tさとい\tサトウ\tサトー\nいちろう\t名詞\t固有名詞\t人名\t名\t*\t*\tいちろう\tイチロウ\tイチロー\n名前\t名詞\t一般\t*\t*\t*\t*\t名前\tナマエ\tナマエ\n砂糖\t名詞\t一般\t*\t*\t*\t*\t砂糖\tサトウ\tサトー\n一\t名詞\t数\t*\t*\t*\t*\t一\tイチ\tイチ\n郎\t名詞\t一般\t*\t*\t*\t*\t郎\tロウ\tロー\n2019\t名詞\t数\t*\t*\t*\t*\t*\n年\t名詞\t接尾\t助数詞\t*\t*\t*\t年\tネン\tネン\n12\t名詞\t数\t*\t*\t*\t*\t*\n月\t名詞\t一般\t*\t*\t*\t*\t月\tツキ\tツキ\n25\t名詞\t数\t*\t*\t*\t*\t*\n日\t名詞\t接尾\t助数詞\t*\t*\t*\t日\tニチ\tニチ\nEOS\n'

In [20]:
f = open('text.txt', 'w') # 書き込みモードで開く
f.write(text) # 引数の文字列をファイルに書き込む
f.close() # ファイルを閉じる